In [1]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import matplotlib.pyplot as plt
import numpy as np
import torch
import pickle

from scripts.smdp_agent import SMDPQLearningAgent
from scripts.intraoption_agent import IntraOptionQLearningAgent
from scripts.taxi_utils import TaxiUtils as tu
from scripts.options import Option
from scripts.policies import MoveTaxiPolicy, EpsilonGreedyPolicy
from scripts.training import Trainer, trainingInspector, test_agent, plot_test_results, compute_decay

## Hyperparameters

In [2]:
def episode_trigger(x):
    if x % 1000 == 0:
        return True
    return False

def process_hyperparameters(hyperparameters):
    hyperparameters.update({
        "eps_decay": compute_decay(
            hyperparameters["eps_start"],
            hyperparameters["eps_end"],
            hyperparameters["frac_episodes_to_decay"],
            hyperparameters["num_episodes"],
            hyperparameters["decay_type"]
        )
    })

    hyperparameters.pop("frac_episodes_to_decay", None)
    return hyperparameters
    

In [3]:
# Top 3 hyperparameter configurations
smdp_hyperparameter_list = [

    process_hyperparameters(_) for _ in  [
    # Best performing hyperparameters
    {
        "num_episodes": 30000,
        "max_return": 8,
        "LR": 1e-3,
        "eps_start": 1,
        "eps_end": 0.005,
        "decay_type": "exponential",
        "frac_episodes_to_decay": 0.7
    }

]]

## Running Experiments

In [4]:
def get_termination(source):
    # ((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination
    termination = [0.5 for i in range(500)]
    
    base = 100*source[0] + 20*source[1]
    idx = tu.LOC_TO_COLOR[source]

    for i in range(4):
        for j in range(5):
            if j!=idx:
                termination[base + j*4 + i] = 1

    return termination
    
initiation_set = [True for i in range(500)]

options = [
    Option(index = i, 
           initiation_set = initiation_set, 
           policy = MoveTaxiPolicy(tu.COLOR_TO_LOC[i]), 
           termination = get_termination(tu.COLOR_TO_LOC[i]), 
           seed=0
    )
    for i in [tu.RED, tu.GREEN, tu.YELLOW, tu.BLUE]
]

In [ ]:
env = gym.make('Taxi-v3', render_mode="rgb_array")
env = RecordVideo(
    env,
    video_folder="backups/taxi-smdp-visualizations",
    name_prefix="eval",
    episode_trigger=episode_trigger
)

smdp_agent = SMDPQLearningAgent(
    state_space=env.observation_space,
    action_space=env.action_space,
    options=options,
    seed=0
)

trainer = Trainer()
smdp_results = test_agent(env, smdp_agent, trainer, smdp_hyperparameter_list, num_experiments=1)
env.close()

# with open("backups/acrobot-plots/acrobot_ddqn_type1_results.pickle", 'wb') as handle:
#     pickle.dump(ddqn_type1_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/lalit/my_venvs/rl_env/lib/python3.10/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/lalit/acads/DA6400/Assignment3/DA6400_RL_PA3/backups/taxi-smdp-visualizations folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


LR 0.001
eps_start 1
eps_end 0.005
decay_type exponential
eps_decay 0.9997477309981986


Training:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▎          | 27247/30000 [02:30<00:09, 302.67it/s, Mean Score=-31.6]

In [ ]:
plot_test_results(smdp_results, [0])

In [ ]:
Qtable = np.reshape(smdp_agent.Qtable, (5,5,5,4,4))
np.argmax(Qtable[:,:,4,3,:], axis=-1)